# Authorisation

This notebook tests granting and retracting authorisations for different levels of medical actor, and assigning medical staff a permission level.

In [1]:
% Set up required to reset environment
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

We initially expect that a diagnosis will have been made by a medical staff member authorised to do so. 

The hospital is aware of various labs, which can be certified to perform various tests. Labs can have existing certifications upon being linked to the hospital, or have certifications added or withdrawn.

Staff members have a permission level i.e. nurse or doctor. Different levels are able to perform different actions. A level can have actions granted or revoked, and a staff member can have their permission level changed.

In [2]:
% File: helperFunctions.pl
list_concat([],L,L).
list_concat([X1|L1],L2,[X1|L3]) :- list_concat(L1,L2,L3).

In [3]:
% File: labInitiates.pl

initially(exp_rule(diagnosed(_,_,_,_,Actor), authorised(diagnosis,Actor))).

initially(certified(Lab,Test)):- initially(lab(_,Lab,L)), member(Test,L).
initiates(certification(Lab,Test,true), certified(Lab,Test),_).
terminates(certification(Lab,Test,false), certified(Lab,Test),_).
initiates(labConnection(_,Lab,L), certified(Lab,Test),_):- member(Test,L).
initiates(labConnection(Name,Lab,L), lab(Name,Lab,L),_).
terminates(labWithdrawal(Lab), lab(_,Lab,_),_).

initiates(newStaff(Name,Actor,Level), medStaff(Name,Actor,Level),_).

terminates(promotion(Actor,NewLevel), medStaff(Name,Actor,OldLevel),T).


initially(authorised(Action,Actor)):- initially(authority(Level,L)), initially(medStaff(_,Actor,Level)), member(Action,L).
initiates(diagnosis(RID,PatID,Test,Result,Actor), diagnosed(RID,PatID,Test,Result,Actor),_).
initiates(newStaff(_,Actor,Level), authorised(Action,Actor),T):- holdsAt(authority(Level,L),T), member(Action,L).
initiates(authorityAdd(Level, LAdd), authority(Level,LNew),T):- holdsAt(authority(Level,LOld),T), list_concat(LOld,LAdd,LNew).


initiates(authorityAdd(Level,L), authorised(Action,Actor),T):- holdsAt(medStaff(_,Actor,Level),T), member(Action,L).
initiates(promotion(Actor,NewLevel), medStaff(Name,Actor,NewLevel),T):- holdsAt(medStaff(Name,Actor,_),T).
initiates(promotion(Actor, NewLevel), authorised(Action,Actor),T):- holdsAt(authority(NewLevel,L),T), member(Action,L).

terminates(oldStaff(Actor,Level), authorised(Action,Actor),T):- holdsAt(authority(Level,L),T), member(Action,L).
terminates(authorityRemove(Level,L), authorised(Action,Actor),T):- holdsAt(medStaff(_,Actor,Level),T), member(Action,L).
terminates(promotion(Actor, NewLevel), authorised(Action,Actor),T):- holdsAt(authority(OldLevel,L1),T), member(Action,L1), holdsAt(authority(NewLevel,L2),T), \+ member(Action,L2).

In [4]:
% File: agents.pl
initially(authority(nurse,[testRequests])).
initially(authority(doctor, [testRequests,diagnosis])).
initially(authority(specialist, [testRequests, diagnosis, brainDeath])).
initially(lab('Otago Blood','OB',[serology])).
initially(lab('Wellington Main', 'WM',[])).
initially(medStaff('Aria Hollens','AH',nurse)).
initially(medStaff('Enola Holmes', 'EH', nurse)).
initially(medStaff('Philip Schmidt', 'PS', doctor)).
happensAtNarrative(authorityAdd(acceptor, [donorDec]),3).
happensAtNarrative(authorityAdd(nurse, [nurseAct1, nurseAct2]),1).
happensAtNarrative(promotion('EH', doctor),2).
happensAtNarrative(authorityRemove(nurse, [nurseAct2]),4).
happensAtNarrative(newStaff('Jane Wald', 'JW', specialist),0).
happensAtNarrative(oldStaff('JW', specialist),2).
happensAtNarrative(newStaff('Gregory MacMillian', 'GM', acceptor),2).
happensAtNarrative(certification('WM', 'HIV',true),2).
happensAtNarrative(labConnection('Auckland Central', AC, ['Hep-C']),3).
happensAtNarrative(certification('WM', 'HIV',false),5).

In [5]:
% File: narrative.pl
happensAtNarrative(diagnosis(7,123,'HIV', pos,'JW'),1).
happensAtNarrative(diagnosis(7,123,'HIV', pos,'AH'),1).


In [6]:
?- holdsAt(authority(doctor,L1),0), holdsAt(authority(OldLevel,L2),0), member(Action,L2), \+ member(Action, L1).

L1 = [ testRequests, diagnosis ], OldLevel = specialist, L2 = [ testRequests, diagnosis, brainDeath ], Action = brainDeath .

In [7]:
?- tick(0).
?- holdsAt(F,0).
?- happensAt(E,0).

true.
F = lab(Otago Blood, OB, [Atom('667525')]) ;
F = lab(Wellington Main, WM, []) ;
F = certified(OB, serology) ;
F = authorised(testRequests, AH) ;
F = authorised(testRequests, EH) ;
F = authorised(testRequests, PS) ;
F = authorised(diagnosis, PS) ;
F = authority(nurse, [Atom('666757')]) ;
F = authority(doctor, [Atom('666757'), Atom('661893')]) ;
F = authority(specialist, [Atom('666757'), Atom('661893'), Atom('667141')]) ;
F = medStaff(Aria Hollens, AH, nurse) ;
F = medStaff(Enola Holmes, EH, nurse) ;
F = medStaff(Philip Schmidt, PS, doctor) ;
F = exp_rule(diagnosed(_3034, _3036, _3038, _3040, _3042), authorised(diagnosis, _3042)) .
E = newStaff(Jane Wald, JW, specialist) .

At time one, we know that the history for patient 123 has already been requested, and at this time, the history is granted.

In [8]:
?- tick(1).
?- holdsAt(F,1).
?- happensAt(E,1).

true.
F = lab(Otago Blood, OB, [Atom('667525')]) ;
F = lab(Wellington Main, WM, []) ;
F = certified(OB, serology) ;
F = authorised(testRequests, AH) ;
F = authorised(testRequests, EH) ;
F = authorised(testRequests, PS) ;
F = authorised(diagnosis, PS) ;
F = authorised(testRequests, JW) ;
F = authorised(diagnosis, JW) ;
F = authorised(brainDeath, JW) ;
F = authority(nurse, [Atom('666757')]) ;
F = authority(doctor, [Atom('666757'), Atom('661893')]) ;
F = authority(specialist, [Atom('666757'), Atom('661893'), Atom('667141')]) ;
F = medStaff(Aria Hollens, AH, nurse) ;
F = medStaff(Enola Holmes, EH, nurse) ;
F = medStaff(Philip Schmidt, PS, doctor) ;
F = medStaff(Jane Wald, JW, specialist) ;
F = exp_rule(diagnosed(_3014, _3016, _3018, _3020, _3022), authorised(diagnosis, _3022)) .
E = authorityAdd(nurse, [Atom('668933'), Atom('669061')]) ;
E = diagnosis(7, 123, HIV, pos, JW) ;
E = diagnosis(7, 123, HIV, pos, AH) .

At time 2, the earlier history request has been answered.

In [9]:
?- tick(2).
?- holdsAt(F,2).
?- happensAt(O,2).
?- happensAtNarrative(P,2).
?- holdsAt(authority(nurse,X),2).

true.
F = lab(Otago Blood, OB, [Atom('667525')]) ;
F = lab(Wellington Main, WM, []) ;
F = certified(OB, serology) ;
F = authorised(testRequests, AH) ;
F = authorised(testRequests, EH) ;
F = authorised(testRequests, PS) ;
F = authorised(diagnosis, PS) ;
F = authorised(testRequests, JW) ;
F = authorised(diagnosis, JW) ;
F = authorised(brainDeath, JW) ;
F = authorised(nurseAct1, AH) ;
F = authorised(nurseAct2, AH) ;
F = authorised(nurseAct1, EH) ;
F = authorised(nurseAct2, EH) ;
F = diagnosed(7, 123, HIV, pos, JW) ;
F = diagnosed(7, 123, HIV, pos, AH) ;
F = authority(nurse, [Atom('666757')]) ;
F = authority(doctor, [Atom('666757'), Atom('661893')]) ;
F = authority(specialist, [Atom('666757'), Atom('661893'), Atom('667141')]) ;
F = authority(nurse, [Atom('666757'), Atom('668933'), Atom('669061')]) .
O = promotion(EH, doctor) ;
O = oldStaff(JW, specialist) ;
O = newStaff(Gregory MacMillian, GM, acceptor) ;
O = certification(WM, HIV, true) ;
O = fulf(diagnosed(7, 123, HIV, pos, JW), authoris

The expectation of an answer has been fulfilled.

In [10]:
?- holdsAt(authority(OldLevel,L1),2), member(Action,L1), holdsAt(authority(NewLevel,L2),2), \+ member(Action,L2).

OldLevel = doctor, L1 = [ testRequests, diagnosis ], Action = diagnosis, NewLevel = nurse, L2 = [ testRequests ] ;
OldLevel = doctor, L1 = [ testRequests, diagnosis ], Action = diagnosis, NewLevel = nurse, L2 = [ testRequests, nurseAct1, nurseAct2 ] ;
OldLevel = specialist, L1 = [ testRequests, diagnosis, brainDeath ], Action = diagnosis, NewLevel = nurse, L2 = [ testRequests ] ;
OldLevel = specialist, L1 = [ testRequests, diagnosis, brainDeath ], Action = diagnosis, NewLevel = nurse, L2 = [ testRequests, nurseAct1, nurseAct2 ] ;
OldLevel = specialist, L1 = [ testRequests, diagnosis, brainDeath ], Action = brainDeath, NewLevel = nurse, L2 = [ testRequests ] ;
OldLevel = specialist, L1 = [ testRequests, diagnosis, brainDeath ], Action = brainDeath, NewLevel = doctor, L2 = [ testRequests, diagnosis ] ;
OldLevel = specialist, L1 = [ testRequests, diagnosis, brainDeath ], Action = brainDeath, NewLevel = nurse, L2 = [ testRequests, nurseAct1, nurseAct2 ] ;
OldLevel = nurse, L1 = [ testReque

In [11]:
?- tick(3).
?- holdsAt(medStaff(Name,ID,Level),3).
?- holdsAt(authorised(Action,Actor),3).
?- holdsAt(authority(Level, Action),3).
?- happensAt(D,3).

true.
Name = Aria Hollens, ID = AH, Level = nurse ;
Name = Philip Schmidt, ID = PS, Level = doctor ;
Name = Jane Wald, ID = JW, Level = specialist ;
Name = Enola Holmes, ID = EH, Level = doctor ;
Name = Gregory MacMillian, ID = GM, Level = acceptor .
Action = testRequests, Actor = AH ;
Action = testRequests, Actor = EH ;
Action = testRequests, Actor = PS ;
Action = diagnosis, Actor = PS ;
Action = nurseAct1, Actor = AH ;
Action = nurseAct2, Actor = AH ;
Action = testRequests, Actor = EH ;
Action = diagnosis, Actor = EH .
Level = nurse, Action = [ testRequests ] ;
Level = doctor, Action = [ testRequests, diagnosis ] ;
Level = specialist, Action = [ testRequests, diagnosis, brainDeath ] ;
Level = nurse, Action = [ testRequests, nurseAct1, nurseAct2 ] .
D = authorityAdd(acceptor, [Atom('668805')]) ;
D = labConnection(Auckland Central, _2976, [Atom('669957')]) .

In [13]:
?- initialiseDEC.
?- run(6).
?- happensAt(B,4).
?- holdsAt(authorised(Action,Actor),4).
?- happensAt(D,5).
?- holdsAt(E,6).

true.
true.
B = authorityRemove(nurse, [Atom('669061')]) ;
B = viol(diagnosed(7, 123, HIV, pos, AH), authorised(diagnosis, AH), 4, authorised(diagnosis, AH)) ;
B = viol(diagnosed(7, 123, HIV, pos, JW), authorised(diagnosis, JW), 4, authorised(diagnosis, JW)) .
Action = testRequests, Actor = AH ;
Action = testRequests, Actor = EH ;
Action = testRequests, Actor = PS ;
Action = diagnosis, Actor = PS ;
Action = nurseAct1, Actor = AH ;
Action = nurseAct2, Actor = AH ;
Action = testRequests, Actor = EH ;
Action = diagnosis, Actor = EH ;
Action = donorDec, Actor = GM .
D = certification(WM, HIV, false) .
E = lab(Otago Blood, OB, [Atom('667525')]) ;
E = lab(Wellington Main, WM, []) ;
E = lab(Auckland Central, _3030, [Atom('669957')]) ;
E = certified(OB, serology) ;
E = certified(_3028, Hep-C) ;
E = authorised(testRequests, AH) ;
E = authorised(testRequests, EH) ;
E = authorised(testRequests, PS) ;
E = authorised(diagnosis, PS) ;
E = authorised(nurseAct1, AH) ;
E = authorised(testRequests, EH) 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>